# VRI anomaly detection bites

This notebook has the aim to study how to detect anomalies in the VRI computed by our models displayed here: https://labs.mosquitoalert.com/MosquitoAlertES/

Data gathered from models bites.

## Requirements

In [1]:
import pandas as pd
from prophet import Prophet
from prophet.plot import seasonality_plot_df
import os
import pandas as pd
import json
from datetime import datetime
import re
import geopandas as gpd
from tqdm import tqdm
from scipy.signal import savgol_filter

Importing plotly failed. Interactive plots will not work.


## Dataset

In [2]:
BITING_DATA_PATH = './biting/'

# Collect all CSV file paths
files = [os.path.join(root, file)
        for root, _, files in os.walk(BITING_DATA_PATH)
        for file in files if file == 'gadm4_monthly.csv']

# Initialize an empty list to hold the data
dfs = []
# Loop through the files
for file in files:
    try:
        dfs.append(pd.read_csv(file))
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Create a DataFrame from the list of data
df = pd.concat(dfs)
del dfs

In [3]:
df

,year,month,gid_4,est,se
0,2014,5,ALA.2.4_2,0.155015,0.719975
1,2014,5,ALA.2.5_2,0.256332,0.869462
2,2014,5,ALA.3.1_2,0.252280,0.864868
3,2014,5,ALB.1.1.1_1,0.541033,0.997685
4,2014,5,ALB.1.1.10_1,0.526849,0.999200
...,...,...,...,...,...
70826,2021,9,ZNC.1_1,0.693000,0.922499
70827,2021,9,ZNC.2_1,0.747000,0.869462
70828,2021,9,ZNC.3_1,0.675000,0.936750
70829,2021,9,ZNC.4_1,0.713000,0.904723


In [4]:
# Rename columns for Prophet
df['ds'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
df.rename(columns={"est": "y"}, inplace=True)

df.sort_values(by=['gid_4', 'ds'], inplace=True, ignore_index=True)

# Keep only values for gid_4, ds, y
df = df[['ds', 'gid_4', 'y']]

# Keep only values for Spanish municipalities
df = df[df['gid_4'].str.startswith('ESP')]
df

,ds,gid_4,y
2990468,2014-01-01,ESP.1.1.1.10_1,0.393110
2990469,2014-02-01,ESP.1.1.1.10_1,0.402229
2990470,2014-03-01,ESP.1.1.1.10_1,0.315096
2990471,2014-04-01,ESP.1.1.1.10_1,0.482270
2990472,2014-05-01,ESP.1.1.1.10_1,0.426545
...,...,...,...
4028456,2024-03-01,ESP.9.1.9.9_1,0.219000
4028457,2024-04-01,ESP.9.1.9.9_1,0.208000
4028458,2024-05-01,ESP.9.1.9.9_1,0.354000
4028459,2024-06-01,ESP.9.1.9.9_1,0.442000


In [5]:
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Function to train a model and detect anomalies for each city
def detect_anomalies_for_city(city_data):
    group_name, city_df = city_data
    if (city_df['y'] == 0).all():  # Skip if all original items are 0
        return None, None

    first_non_zero = city_df[city_df["y"] != 0].iloc[0]
    holidays_df = city_df[(city_df['y']==0) & (city_df['ds'] < first_non_zero['ds'])]['ds'].reset_index()
    holidays_df['holiday'] = 'no-prediction-yet'
    
    # Step 3: Initialize Prophet with logistic growth
    model = Prophet(growth='logistic', yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False, holidays=holidays_df[['ds','holiday']])
    city_df.loc[:,'cap'] = 1
    city_df.loc[:,'floor'] = 0
    model.fit(city_df)
    
    # Make predictions for historical data (no future periods)
    future = model.make_future_dataframe(periods=0)
    future['cap'] = 1  # Ensure the future data has the cap
    future['floor'] = 0  # Ensure the future data has the floor
    forecast = model.predict(future)

    forecast['fact'] = city_df['y'].reset_index(drop = True)

    forecast['anomaly'] = 0
    forecast.loc[forecast['fact'] > forecast['yhat_upper'], 'anomaly'] = 1
    forecast.loc[forecast['fact'] < forecast['yhat_lower'], 'anomaly'] = -1

     #anomaly importances
    forecast['importance'] = 0.0
    forecast.loc[forecast['anomaly'] ==1, 'importance'] = \
        (forecast['fact'] - forecast['yhat_upper'])/forecast['fact']
    forecast.loc[forecast['anomaly'] ==-1, 'importance'] = \
        (forecast['yhat_lower'] - forecast['fact'])/forecast['fact']

    # Merge forecast with the original data
    city_df_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance']]
    result_df = city_df[['gid_4', 'ds']].merge(city_df_forecast, on='ds', how='left')

    # Seasonality component
    df_w = seasonality_plot_df(m=model, ds=pd.date_range(start='2017-01-01', periods=365))
    seas_df = model.predict_seasonal_components(df_w)
    yearly_df = seas_df['yearly'].reset_index()
    yearly_df.loc[:,'gid_4'] = city_df.iloc[0]['gid_4']

    return result_df, yearly_df

In [6]:
from concurrent.futures import ProcessPoolExecutor
import os
import math

# Apply the anomaly detection for each city in parallel
with ProcessPoolExecutor(max_workers=math.floor(max(os.cpu_count() * 0.8, 1))) as executor:
    results = list(
        tqdm(
            executor.map(
                detect_anomalies_for_city,
                df.groupby('gid_4')
            ), 
            total=len(
                df['gid_4'].unique()
            )
        )
    )


100%|███████████████████████████████████████| 8205/8205 [02:32<00:00, 53.67it/s]


In [7]:
# Combine the results for all cities
result_df = df.merge(
    pd.concat([arr[0] for arr in results if arr is not None]),
    on=['gid_4', 'ds'], 
    how='left'
)
# Setting a 0 for the prediction value that hasn't been predicted because was all 0.
result_df[['yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance']] = result_df[['yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance']].fillna(0)
yearly_seasonality_df = pd.concat([arr[1] for arr in results if arr is not None])

In [8]:
result_df.to_csv('spain_activty_anomaly_bites.csv', index=False)
yearly_seasonality_df.to_csv('spain_seasonality_bites.csv', index=False)

## Part 2

In [2]:
result_df = pd.read_csv('./spain_activty_anomaly_bites.csv')
yearly_seasonality_df = pd.read_csv('./spain_seasonality_bites.csv')

In [9]:
result_df

,ds,gid_4,y,yhat,yhat_lower,yhat_upper,trend,anomaly,importance
0,2014-01-01,ESP.1.1.1.10_1,0.393110,0.322870,0.250874,0.392607,0.528668,1,0.001281
1,2014-02-01,ESP.1.1.1.10_1,0.402229,0.326708,0.257482,0.394605,0.527618,1,0.018955
2,2014-03-01,ESP.1.1.1.10_1,0.315096,0.365371,0.298259,0.437156,0.526669,0,0.000000
3,2014-04-01,ESP.1.1.1.10_1,0.482270,0.420077,0.357624,0.490649,0.525618,0,0.000000
4,2014-05-01,ESP.1.1.1.10_1,0.426545,0.488816,0.422413,0.554239,0.524601,0,0.000000
...,...,...,...,...,...,...,...,...,...
1037988,2024-03-01,ESP.9.1.9.9_1,0.219000,0.200083,0.142992,0.260890,0.337588,0,0.000000
1037989,2024-04-01,ESP.9.1.9.9_1,0.208000,0.235622,0.178164,0.296882,0.336815,0,0.000000
1037990,2024-05-01,ESP.9.1.9.9_1,0.354000,0.345233,0.288684,0.400505,0.336067,0,0.000000
1037991,2024-06-01,ESP.9.1.9.9_1,0.442000,0.473369,0.418315,0.531617,0.335295,0,0.000000


In [10]:
current_status_df = result_df.sort_values(
    by=['gid_4', 'ds']
).groupby('gid_4').apply(lambda x: x.iloc[-1])[['y', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance', 'ds']]

/tmp/ipykernel_462799/1002646056.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).groupby('gid_4').apply(lambda x: x.iloc[-1])[['y', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance', 'ds']]


In [11]:
current_status_df.reset_index(inplace=True)

In [12]:
current_status_df.rename(columns={'ds': 'last_update'}, inplace=True)
current_status_df['gid_4'] = current_status_df['gid_4'].astype(str)

### Load shapefiles & save geopackage

In [13]:
peninsula_gdf = gpd.read_file('lineas_limite/SHP_ETRS89/recintos_municipales_inspire_peninbal_etrs89')
peninsula_gdf = peninsula_gdf.to_crs(epsg=4326)

canarias_gdf = gpd.read_file('lineas_limite/SHP_REGCAN95/recintos_municipales_inspire_canarias_regcan95')
canarias_gdf = canarias_gdf.to_crs(epsg=4326)

spain_gdf = gpd.GeoDataFrame(pd.concat([peninsula_gdf, canarias_gdf], ignore_index=True))
spain_gdf['NAMEUNIT'] = spain_gdf['NAMEUNIT'].str.split('/').str[0]

In [14]:
peninsula_ccaa_gdf = gpd.read_file('lineas_limite/SHP_ETRS89/recintos_autonomicas_inspire_peninbal_etrs89')
peninsula_ccaa_gdf = peninsula_ccaa_gdf.to_crs(epsg=4326)

canarias_ccaa_gdf = gpd.read_file('lineas_limite/SHP_REGCAN95/recintos_autonomicas_inspire_canarias_regcan95')
canarias_ccaa_gdf = canarias_ccaa_gdf.to_crs(epsg=4326)

spain_ccaa_gdf = gpd.GeoDataFrame(pd.concat([peninsula_ccaa_gdf, canarias_ccaa_gdf], ignore_index=True))
spain_ccaa_gdf['NAMEUNIT'] = spain_ccaa_gdf['NAMEUNIT'].str.split('/').str[0]

In [15]:
gadm4_gdf = gpd.read_file('gadm41_ESP.gpkg', layer='ADM_ADM_4')

In [16]:
gadm4_gdf['geometry'] = gadm4_gdf.representative_point()

In [17]:
municipalities_gdf = gpd.sjoin(spain_gdf, gadm4_gdf, how="left")[[
    'GID_4', 'NATCODE', 'NAMEUNIT', 'CODNUT2', 'geometry'
]]

In [18]:
gdf = municipalities_gdf[['GID_4', 'NATCODE', 'NAMEUNIT', 'CODNUT2', 'geometry']].merge(
    spain_ccaa_gdf[['NAMEUNIT', 'CODNUT2']].rename(columns={'NAMEUNIT': 'NAMEUNIT_NUT2'}),
    on='CODNUT2',
    how='inner'
)
gdf['NATCODE'] = gdf['NATCODE'].astype(int)

In [19]:
gdf

,GID_4,NATCODE,NAMEUNIT,CODNUT2,geometry,NAMEUNIT_NUT2
0,ESP.17.1.5.3_1,34033333022,Degaña,ES12,"MULTIPOLYGON (((-6.6574 42.96745, -6.64737 42....",Principado de Asturias
1,ESP.17.1.3.4_1,34033333023,El Franco,ES12,"MULTIPOLYGON (((-6.87709 43.56358, -6.87705 43...",Principado de Asturias
2,ESP.17.1.4.2_1,34033333024,Gijón,ES12,"MULTIPOLYGON (((-5.81929 43.50727, -5.8184 43....",Principado de Asturias
3,ESP.17.1.1.6_1,34033333025,Gozón,ES12,"MULTIPOLYGON (((-5.91545 43.60853, -5.91537 43...",Principado de Asturias
4,ESP.17.1.8.4_1,34033333026,Grado,ES12,"POLYGON ((-6.20021 43.18357, -6.20121 43.1856,...",Principado de Asturias
...,...,...,...,...,...,...
8335,ESP.14.2.1.3_1,34053838003,Alajeró,ES70,"MULTIPOLYGON (((-17.22374 28.0254, -17.22373 2...",Canarias
8336,ESP.14.2.1.4_1,34053838004,Arafo,ES70,"POLYGON ((-16.48414 28.33504, -16.48377 28.336...",Canarias
8337,ESP.14.2.1.5_1,34053838005,Arico,ES70,"MULTIPOLYGON (((-16.47283 28.10376, -16.47287 ...",Canarias
8338,ESP.14.2.1.6_1,34053838006,Arona,ES70,"MULTIPOLYGON (((-16.6964 28.00131, -16.69639 2...",Canarias


In [20]:
current_gdf = gdf.merge(current_status_df.rename(columns={'gid_4': 'GID_4'}), on='GID_4')
current_gdf.set_index('NATCODE', inplace=True)
current_gdf.drop(columns=['GID_4'], inplace=True)

In [21]:
historic_gdf = gpd.GeoDataFrame(
    result_df.merge(
        gdf[['NATCODE', 'GID_4']].rename(columns={'GID_4': 'gid_4'}),
        on='gid_4',
        how='inner'
    ).drop(columns=['gid_4']),
    geometry=None
)

In [22]:
gpk_path = 'output_bites.gpkg'
# Save the GeoPandas DataFrame (geometries)
current_gdf.to_file(gpk_path, layer='geometries', driver="GPKG")
historic_gdf.to_file(gpk_path, layer='histories', driver="GPKG")

In [23]:
gpd.GeoDataFrame(yearly_seasonality_df.merge(
        gdf[['NATCODE', 'GID_4']].rename(columns={'GID_4': 'gid_4'}),
        on='gid_4',
        how='inner'
    ).drop(columns=['gid_4']), geometry=None).to_file(gpk_path, layer='seasonality', driver="GPKG")